# TFRS

This notebook takes the master data that was prepared in the "data_preprocessing" notebook, creates a test holdout set from the 20% of the data that the same split are used for other experimentations for consistency.

The model is created by TensorFlow Recommenders.

In [1]:
!pip install -q tensorflow-recommenders


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

from typing import Dict, Text

## Train-test Split

The train test split is done by only taking the 20% of the data as the test holdout set. For making sure the train and test data is consistent in all experiments, the following test holdout split will be the same for each experiment.

It is an important detail that the split is done in a stratified way to ensure that the user rankings will be splitted as evenly as possible.

In [3]:
#Do not load the "timestamp" column since it is not needed for building the recommender engine
df = pd.read_csv('data/master_data.zip', compression="zip")[["userId", "movieId", "rating"]]


Data is transformed into TensorFlow dataset format 

In [4]:
trainset = tf.data.Dataset.from_tensor_slices(df.values)

trainset = trainset.map(lambda x: {
    "user_id": tf.as_string(tf.cast(x[0], tf.int32))  ,
    "movie_id": tf.as_string(tf.cast(x[1], tf.int32)) ,
    "rating": tf.cast(x[2], tf.float32)
})


Unique user and movie IDs are determined for embedding generation

In [7]:
movie_ids = trainset.batch(1_000_000).map(lambda x: x["movie_id"])
user_ids = trainset.batch(1_000_000).map(lambda x: x["user_id"])

import joblib
unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))
joblib.dump(unique_movie_ids, "data/unique_movie_ids.pkl")
#unique_movie_ids = joblib.load("data/unique_movie_ids.pkl")
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
joblib.dump(unique_user_ids, "data/unique_user_ids.pkl")

['data/unique_user_ids.pkl']

## Model Build

In [8]:
class ModelRanking(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dims = 32

    # User embeddings
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dims)
    ])

    # Movie Embeddings
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dims)
    ])

    # Predictions
    self.ratings = tf.keras.Sequential([
      # multiple dense layers
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Ratings in output layer
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):
    user_id, movie_id = inputs

    user_embed = self.user_embeddings(user_id)
    movie_embed = self.movie_embeddings(movie_id)

    return self.ratings(tf.concat([user_embed, movie_embed], axis=1))
  
  
  
class ModelMovielens(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = ModelRanking()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("rating")
    rating_predictions = self(features)

    # Compute loss and metric
    return self.task(labels=labels, predictions=rating_predictions)

In [9]:
cached_train = trainset.shuffle(100_000).batch(8192).cache()

In [10]:
model = ModelMovielens()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005))

In [11]:
history = model.fit(cached_train, epochs=20)

Epoch 1/20
1362/1362 [==============================] - 190s 138ms/step - root_mean_squared_error: 0.8675 - loss: 0.7524 - regularization_loss: 0.0000e+00 - total_loss: 0.7524 - root_mean_squared_error: 0.8688 -
Epoch 2/20
1362/1362 [==============================] - 66s 49ms/step - root_mean_squared_error: 0.7893 - loss: 0.6230 - regularization_loss: 0.0000e+00 - total_loss: 0.6230
Epoch 3/20
1362/1362 [==============================] - 67s 49ms/step - root_mean_squared_error: 0.7712 - loss: 0.5948 - regularization_loss: 0.0000e+00 - total_loss: 0.5948
Epoch 4/20
1362/1362 [==============================] - 71s 52ms/step - root_mean_squared_error: 0.7600 - loss: 0.5776 - regularization_loss: 0.0000e+00 - total_loss: 0.57762s - root_mean_squared_error: 0.7601 -
Epoch 5/20
1362/1362 [==============================] - 68s 50ms/step - root_mean_squared_error: 0.7483 - loss: 0.5600 - regularization_loss: 0.0000e+00 - total_loss: 0.5600
Epoch 6/20
1362/1362 [==============================] 

In [14]:
model.save_weights('data/recommendation_model_weights.h5')


In [21]:
test_ratings = {}
test_movie_ids = ["0", "11", "199"]

user_id_test = "42"


for movie_id in test_movie_ids:
  movie_name = movie_dict.get(int(movie_id))
  test_ratings[movie_name] = model({
      "user_id": np.array([user_id_test]),
      "movie_id": np.array([movie_id])
  })

print("Recommendations:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Recommendations:
Toy Story (1995): [[3.506403]]
Jefferson in Paris (1995): [[2.778992]]
Dracula: Dead and Loving It (1995): [[1.8170128]]
